# Bind
- Runnable Chain에 이전 runnable의 출력이나 사용자 입력에 없는 값을 입력해야 할 때 사용한다
- OpenAI Function을 모델에 연결하거나, Tool을 연결할 때 많이 사용한다.

In [2]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_community.chat_models import ChatOllama

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            # 대수 기호를 사용하여 다음 방정식을 작성한 다음 풀이하세요.
            "Write out the following equation using algebraic symbols then solve it. "
            "Use the format\n\nEQUATION:...\nSOLUTION:...\n\n",
        ),
        (
            "human",
            "{equation_statement}",  # 사용자가 입력한 방정식 문장을 변수로 받습니다.
        ),
    ]
)
model = ChatOllama(model="gemma:7b")

# 방정식 문장을 입력받아 프롬프트에 전달하고, 모델에서 생성된 결과를 문자열로 파싱합니다.
runnable = (
    {"equation_statement": RunnablePassthrough()} | prompt | model | StrOutputParser()
)

# 예시 방정식 문장을 입력하여 결과를 출력합니다.
print(runnable.invoke("x raised to the third plus seven equals 12"))


EQUATION: x^3 + 7 = 12

SOLUTION: x^3 = 12 - 7 = 5

Therefore, x = 5^(1/3).


In [5]:
runnable = (
    {"equation_statement": RunnablePassthrough()} 
    | prompt 
    | model.bind(stop=["SOLUTION"]) 
    | StrOutputParser() # SOLUTION에서 토큰 생성을 멈추도록 model에 stop 파라미터를 바인딩한다.
)
# 예시 방정식 문장을 입력하여 결과를 출력합니다.
print(runnable.invoke("x raised to the third plus seven equals 12"))

EQUATION: x^3 + 7 = 12


